In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import random
import math
import re

In [5]:
#functions dictionary 
def smart_open(fname, mode = 'r'):
    if fname.endswith('.gz'):
        import gzip
        # Using max compression (9) by default seems to be slow.                                
        # Let's try using the fastest.                                                          
        return gzip.open(fname, mode, 1)
    else:
        return open(fname, mode)

def tokanizer(list):
  tokens =[]
  for s in list:
      for i in s:
          tokens.append(i.split(' '))
  return tokens

def randint(b, a=0):
    return random.randint(a,b)

def uniq(seq, idfun=None):
    # order preserving                                                                          
    if idfun is None:
        def idfun(x): return x
    seen = {}
    result = []
    for item in seq:
        marker = idfun(item)
        # in old Python versions:                                                               
        # if seen.has_key(marker)                                                               
        # but in new ones:                                                                      
        if marker in seen: continue
        seen[marker] = 1
        result.append(item)
    return result
    
def sort_dict(myDict, byValue=False, reverse=False):
    if byValue:
        items = myDict.items()
        items.sort(key = operator.itemgetter(1), reverse=reverse)
    else:
        items = sorted(myDict.items())
    return items

def max_dict(myDict, byValue=False):
    if byValue:
        skey=lambda x:x[1]
    else:
        skey=lambda x:x[0]
    return max(myDict.items(), key=skey)


def min_dict(myDict, byValue=False):
    if byValue:
        skey=lambda x:x[1]
    else:
        skey=lambda x:x[0]
    return min(myDict.items(), key=skey)

def paragraphs(lines, is_separator=lambda x : x == '\n', joiner=''.join):
    paragraph = []
    for line in lines:
        if is_separator(line):
            if paragraph:
                yield joiner(paragraph)
                paragraph = []
        else:
            paragraph.append(line)
    if paragraph:
        yield joiner(paragraph)

def isASCII(word):
    try:
        word = word.decode("ascii")
        return True
    except UnicodeEncodeError :
        return False
    except UnicodeDecodeError:
        return False


def intersect(x, y):
    return [z for z in x if z in y]

# Mapping Windows CP1252 Gremlins to Unicode
# from http://effbot.org/zone/unicode-gremlins.htm
cp1252 = {
    # from http://www.microsoft.com/typography/unicode/1252.htm
    u"\x80": u"\u20AC", # EURO SIGN
    u"\x82": u"\u201A", # SINGLE LOW-9 QUOTATION MARK
    u"\x83": u"\u0192", # LATIN SMALL LETTER F WITH HOOK
    u"\x84": u"\u201E", # DOUBLE LOW-9 QUOTATION MARK
    u"\x85": u"\u2026", # HORIZONTAL ELLIPSIS
    u"\x86": u"\u2020", # DAGGER
    u"\x87": u"\u2021", # DOUBLE DAGGER
    u"\x88": u"\u02C6", # MODIFIER LETTER CIRCUMFLEX ACCENT
    u"\x89": u"\u2030", # PER MILLE SIGN
    u"\x8A": u"\u0160", # LATIN CAPITAL LETTER S WITH CARON
    u"\x8B": u"\u2039", # SINGLE LEFT-POINTING ANGLE QUOTATION MARK
    u"\x8C": u"\u0152", # LATIN CAPITAL LIGATURE OE
    u"\x8E": u"\u017D", # LATIN CAPITAL LETTER Z WITH CARON
    u"\x91": u"\u2018", # LEFT SINGLE QUOTATION MARK
    u"\x92": u"\u2019", # RIGHT SINGLE QUOTATION MARK
    u"\x93": u"\u201C", # LEFT DOUBLE QUOTATION MARK
    u"\x94": u"\u201D", # RIGHT DOUBLE QUOTATION MARK
    u"\x95": u"\u2022", # BULLET
    u"\x96": u"\u2013", # EN DASH
    u"\x97": u"\u2014", # EM DASH
    u"\x98": u"\u02DC", # SMALL TILDE
    u"\x99": u"\u2122", # TRADE MARK SIGN
    u"\x9A": u"\u0161", # LATIN SMALL LETTER S WITH CARON
    u"\x9B": u"\u203A", # SINGLE RIGHT-POINTING ANGLE QUOTATION MARK
    u"\x9C": u"\u0153", # LATIN SMALL LIGATURE OE
    u"\x9E": u"\u017E", # LATIN SMALL LETTER Z WITH CARON
    u"\x9F": u"\u0178", # LATIN CAPITAL LETTER Y WITH DIAERESIS
}

def fix_cp1252codes(text):
    # map cp1252 gremlins to real unicode characters
    if re.search(u"[\x80-\x9f]", text):
        def fixup(m):
            s = m.group(0)
            return cp1252.get(s, s)
        if isinstance(text, type("")):
            # make sure we have a unicode string
            text = unicode(text, "iso-8859-1")
        text = re.sub(u"[\x80-\x9f]", fixup, text)
    return text

def clean_utf8(text):
    return filter(lambda x : x > '\x1f' and x < '\x7f', text)

def pairs(iterable, overlapping=False):
    iterator = iterable.__iter__()
    token = iterator.next()
    i = 0
    for lookahead in iterator:
        if overlapping or i % 2 == 0: 
            yield (token, lookahead)
        token = lookahead
        i += 1
    if i % 2 == 0:
        yield (token, None)

def frange(start, end=None, inc=None):
    "A range function, that does accept float increments..."

    if end == None:
        end = start + 0.0
        start = 0.0

    if inc == None:
        inc = 1.0

    L = []
    while 1:
        next = start + len(L) * inc
        if inc > 0 and next >= end:
            break
        elif inc < 0 and next <= end:
            break
        L.append(next)
        
    return L

def softmax(values):
    a = max(values)
    Z = 0.0
    for v in values:
        Z += math.exp(v - a)
    sm = [math.exp(v-a) / Z for v in values]
    return sm

In [6]:
import sys
from getopt import getopt

def load_annotation(gold_file):
    source_sentences = []
    gold_edits = []
    fgold = smart_open(gold_file, 'r')
    puffer = fgold.read()
    fgold.close()
    #puffer = puffer.decode('utf8')
    for item in paragraphs(puffer.splitlines(True)):
        item = item.splitlines(False)
        sentence = [line[2:].strip() for line in item if line.startswith('S ')]
        assert sentence != []
        annotations = {}
        for line in item[1:]:
            if line.startswith('I ') or line.startswith('S '):
                continue
            assert line.startswith('A ')
            line = line[2:]
            fields = line.split('|||')
            #print(fields)
            start_offset = int(fields[0].split()[0])
            end_offset = int(fields[0].split()[1])
            etype = fields[1]
            if etype == 'noop':
                start_offset = -1
                end_offset = -1
            corrections =  [c.strip() if c != '-NONE-' else '' for c in fields[2].split('||')]
            # NOTE: start and end are *token* offsets
            original = ' '.join(' '.join(sentence).split()[start_offset:end_offset])
            if len(fields) > 4:
              annotator = int(fields[5])
            if annotator not in annotations.keys():
                annotations[annotator] = []
            annotations[annotator].append((start_offset, end_offset, original, corrections))
        tok_offset = 0
        for this_sentence in sentence:
            tok_offset += len(this_sentence.split())
            source_sentences.append(this_sentence)
            this_edits = {}
            for annotator, annotation in annotations.items():
                this_edits[annotator] = [edit for edit in annotation if edit[0] <= tok_offset and edit[1] <= tok_offset and edit[0] >= 0 and edit[1] >= 0]
            if len(this_edits) == 0:
                this_edits[0] = []
            gold_edits.append(this_edits)
    return (source_sentences, gold_edits)

# **Train Data Preprocessing**

In [7]:
m2_list_sentance=[]
m2_list_operation=[]

m2_list_sentance.append(load_annotation('/content/QALB-Train2014.m2')[0])
m2_list_operation.append(load_annotation('/content/QALB-Train2014.m2')[1])


In [9]:
tokens =[]
i=0
for i in m2_list_sentance:
    tokens = tokanizer(m2_list_sentance)
print(tokens[0])
len(tokens)

['الى', 'التعليق', 'رقم', '2', 'اكيد', 'ان', 'لحكام', 'العرب', 'والمسلمين', 'مسؤولية', 'يتمثل', 'ادناها', 'في', 'استدعاء', 'السفراء', 'في', 'الصين', 'للتشاور', '.', 'فليتهم', 'يفعلونها', 'ولو', 'مرة', '.', 'ولنا', 'نحن', 'كشعوب', 'مسؤولية', 'كذالك', 'تتمثل', 'في', 'مساندة', 'اخواننا', 'في', 'الصين', 'بمقاطعة', 'البضائع', 'الصينينة', 'وليتنا', 'نفعلها', 'ولو', 'ثلاتة', 'اشهر', '.']


19411

In [10]:
def extract(operation):
  input=[]
  correct=[]

  for i in range(len(operation)):
          if (operation[i][0]!=operation[i][1]):  
              input.append(operation[i][2])
              correct.append(operation[i][3][0])
  return (input , correct)

In [12]:
list_correction=[]
list_input=[]
for i in range(len(tokens)):
  list_input.append((extract(m2_list_operation[0][i][0])[0]))
  list_correction.append((extract(m2_list_operation[0][i][0])[1]))

In [14]:
# Preprocessing regular expressions
import re
def process(sent):
    # sent=sent.lower()
    # sent=re.sub(r'[^0-9]','',sent)

    #sent=sent.replace(' ','\n')
    sent=sent.replace('ـ','')
    sent=sent.replace('-','')
    sent=sent.replace(')','')
    sent=sent.replace('(','')
    sent=sent.replace('*','')
    sent=sent.replace(',','')
    sent=sent.replace('/','')
    sent=sent.replace('&','')
    sent=sent.replace('amp','')
    sent=sent.replace('_','')
    sent=sent.replace('ّ','')
    sent=sent.replace('ً','')
    sent=sent.replace('%','')
    sent=sent.replace(';','')
    sent=sent.replace('ِ','')
    sent=sent.replace('@','')
    sent=sent.replace('ګ','')
    sent=sent.replace( 'ٍ','')
    sent=sent.replace('٪','')
    sent=sent.replace('ٱ','')
    sent=sent.replace('ُ','')
    sent=sent.replace(':','')
    sent=sent.replace('؛','')
    sent=sent.replace('[','')
    sent=sent.replace(']','')
    sent=sent.replace('=','')
    sent=sent.replace('^','')
    sent=sent.replace('?','')
    sent=sent.replace('{','')
    sent=sent.replace('}','')
    sent=sent.replace('g','')
    sent=sent.replace('t','')
    sent=sent.replace('a','')
    sent=sent.replace('b','')
    sent=sent.replace('c','')
    sent=sent.replace('d','')
    sent=sent.replace('s','')
    sent=sent.replace('f','')
    sent=sent.replace('h','')
    sent=sent.replace('j','')
    sent=sent.replace('k','')
    sent=sent.replace('l','')
    sent=sent.replace('q','')
    sent=sent.replace('t','')
    sent=sent.replace('a','')
    sent=sent.replace('b','')
    sent=sent.replace('c','')
    sent=sent.replace('d','')
    sent=sent.replace('s','')
    sent=sent.replace('f','')
    sent=sent.replace('h','')
    sent=sent.replace('j','')
    sent=sent.replace('k','')
    sent=sent.replace('l','')
    sent=sent.replace('q','')
    sent=sent.replace('w','')
    sent=sent.replace('e','')
    sent=sent.replace('r','')
    sent=sent.replace('y','')
    sent=sent.replace('u','')
    sent=sent.replace('i','')
    sent=sent.replace('o','')
    sent=sent.replace('p','')
    sent=sent.replace('z','')
    sent=sent.replace('x','')
    sent=sent.replace('v','')
    sent=sent.replace('n','')
    sent=sent.replace('m','')
    sent=sent.replace('\\','')
    sent=sent.replace('$','')
    sent=sent.replace('`','')
    sent=sent.replace('~','')
    sent=sent.replace('|','')
    sent=sent.replace('گ','')
    sent=sent.replace("'",'')
    # sent=sent.replace('ُ','')
    # sent=sent.replace(':','')
    # sent=sent.replace('ُ','')
    # sent=sent.replace(':','')
    # sent=sent.replace('ُ','')
    # sent=sent.replace(':','')
    # sent=sent.replace('ُ','')
    # sent=sent.replace(':','')
    # sent=sent.replace('ُ','')
    # sent=sent.replace(':','')

    return sent   

In [15]:
#applying the preproccesing &separating the words again
input_sents=[]
for i in range(len(list_input)):
    input_sent =[process(x) for x in list_input[i]]
    for line in input_sent :
            input_sents.append(line)
#print("\n".join(target[:4]))
print("Number of items:",len(input_sents))
input_sents[0]

Number of items: 207479


'الى'

In [16]:
#applying the preproccesing &separating the words again
target_sent=[]
for i in range(len(list_correction)):
    linesout =[process(x) for x in list_correction[i]]
    for line in linesout :
            target_sent.append(line)    
#print("\n".join(target[:4]))
print("Number of items:",len(target_sent))
target_sent[0:4]

Number of items: 207479


['إلى', 'أكيد', 'أن', 'للحكام']

In [17]:
#char indexing
char_set = list('+#’ "‘ی،.؟!يـابتةثجپحخدذرزسشصضطظعڤغفقكلمنهوىيءآۆأؤإڕئک١۱٢٣٤٥٦٧٨٩۹٠0123456789')
char2int = { char_set[x]:x for x in range(len(char_set)) }
int2char = { char2int[x]:x for x in char_set }
print(char2int)
print(int2char)

{'+': 0, '#': 1, '’': 2, ' ': 3, '"': 4, '‘': 5, 'ی': 6, '،': 7, '.': 8, '؟': 9, '!': 10, 'ي': 44, 'ـ': 12, 'ا': 13, 'ب': 14, 'ت': 15, 'ة': 16, 'ث': 17, 'ج': 18, 'پ': 19, 'ح': 20, 'خ': 21, 'د': 22, 'ذ': 23, 'ر': 24, 'ز': 25, 'س': 26, 'ش': 27, 'ص': 28, 'ض': 29, 'ط': 30, 'ظ': 31, 'ع': 32, 'ڤ': 33, 'غ': 34, 'ف': 35, 'ق': 36, 'ك': 37, 'ل': 38, 'م': 39, 'ن': 40, 'ه': 41, 'و': 42, 'ى': 43, 'ء': 45, 'آ': 46, 'ۆ': 47, 'أ': 48, 'ؤ': 49, 'إ': 50, 'ڕ': 51, 'ئ': 52, 'ک': 53, '١': 54, '۱': 55, '٢': 56, '٣': 57, '٤': 58, '٥': 59, '٦': 60, '٧': 61, '٨': 62, '٩': 63, '۹': 64, '٠': 65, '0': 66, '1': 67, '2': 68, '3': 69, '4': 70, '5': 71, '6': 72, '7': 73, '8': 74, '9': 75}
{0: '+', 1: '#', 2: '’', 3: ' ', 4: '"', 5: '‘', 6: 'ی', 7: '،', 8: '.', 9: '؟', 10: '!', 44: 'ي', 12: 'ـ', 13: 'ا', 14: 'ب', 15: 'ت', 16: 'ة', 17: 'ث', 18: 'ج', 19: 'پ', 20: 'ح', 21: 'خ', 22: 'د', 23: 'ذ', 24: 'ر', 25: 'ز', 26: 'س', 27: 'ش', 28: 'ص', 29: 'ض', 30: 'ط', 31: 'ظ', 32: 'ع', 33: 'ڤ', 34: 'غ', 35: 'ف', 36: 'ق', 37: 'ك', 3

In [18]:
#adding additional codes that then used in creating the dataset
count = len(char_set)
codes = ["\t","\n"]
for i in range(len(codes)):
    code = codes[i] #i=0,code=\t
    char2int[code]=count
    int2char[count]=code
    count+=1
print(char2int)
print(int2char)

{'+': 0, '#': 1, '’': 2, ' ': 3, '"': 4, '‘': 5, 'ی': 6, '،': 7, '.': 8, '؟': 9, '!': 10, 'ي': 44, 'ـ': 12, 'ا': 13, 'ب': 14, 'ت': 15, 'ة': 16, 'ث': 17, 'ج': 18, 'پ': 19, 'ح': 20, 'خ': 21, 'د': 22, 'ذ': 23, 'ر': 24, 'ز': 25, 'س': 26, 'ش': 27, 'ص': 28, 'ض': 29, 'ط': 30, 'ظ': 31, 'ع': 32, 'ڤ': 33, 'غ': 34, 'ف': 35, 'ق': 36, 'ك': 37, 'ل': 38, 'م': 39, 'ن': 40, 'ه': 41, 'و': 42, 'ى': 43, 'ء': 45, 'آ': 46, 'ۆ': 47, 'أ': 48, 'ؤ': 49, 'إ': 50, 'ڕ': 51, 'ئ': 52, 'ک': 53, '١': 54, '۱': 55, '٢': 56, '٣': 57, '٤': 58, '٥': 59, '٦': 60, '٧': 61, '٨': 62, '٩': 63, '۹': 64, '٠': 65, '0': 66, '1': 67, '2': 68, '3': 69, '4': 70, '5': 71, '6': 72, '7': 73, '8': 74, '9': 75, '\t': 76, '\n': 77}
{0: '+', 1: '#', 2: '’', 3: ' ', 4: '"', 5: '‘', 6: 'ی', 7: '،', 8: '.', 9: '؟', 10: '!', 44: 'ي', 12: 'ـ', 13: 'ا', 14: 'ب', 15: 'ت', 16: 'ة', 17: 'ث', 18: 'ج', 19: 'پ', 20: 'ح', 21: 'خ', 22: 'د', 23: 'ذ', 24: 'ر', 25: 'ز', 26: 'س', 27: 'ش', 28: 'ص', 29: 'ض', 30: 'ط', 31: 'ظ', 32: 'ع', 33: 'ڤ', 34: 'غ', 35: 'ف',

In [19]:
# create  dataset
target_sents = [] 

for line in target_sent:
        output = '\t' + line + '\n'
        target_sents.append(output)

print(target_sents[:4])

['\tإلى\n', '\tأكيد\n', '\tأن\n', '\tللحكام\n']


In [20]:
print("LEN OF input_sents:",len(input_sents))
print("LEN OF target_sents:",len(target_sents))
print("input_texts",input_sents[:10])
print("target_texts",target_sents[:10])

LEN OF input_sents: 207479
LEN OF target_sents: 207479
input_texts ['الى', 'اكيد', 'ان', 'لحكام', 'ادناها', '.', 'كذالك', 'اخواننا', 'الصينينة', 'ثلاتة']
target_texts ['\tإلى\n', '\tأكيد\n', '\tأن\n', '\tللحكام\n', '\tأدناها\n', '\t!\n', '\tكذلك\n', '\tإخواننا\n', '\tالصينية\n', '\tثلاثة\n']


In [21]:
max_enc_len = max([len(x) for x in input_sents])
max_dec_len = max([len(x) for x in target_sents])

print("Max Enc Len:",max_enc_len)
print("Max Dec Len:",max_dec_len)

num_samples = len(input_sents)

encoderInput_data = np.zeros( (num_samples , max_enc_len , len(char_set)),dtype='float32' )
decoderInput_data = np.zeros( (num_samples , max_dec_len , len(char_set)+2),dtype='float32' )
decoderTarget_data = np.zeros( (num_samples , max_dec_len , len(char_set)+2),dtype='float32' )

print("CREATED ZERO VECTORS")
print(encoderInput_data.shape)

#to cover the enumration issue 
input_sents_enc = [] 
target_sents_dec = [] 

for line in input_sents:
        input_sents_enc.append(line)

for line in target_sents:
        target_sents_dec.append(line)

print("input_sents_enc",len(input_sents_enc))
print("target_sents_dec",len(target_sents_dec))
  

for i,(input_sents_enc,target_sents_dec) in enumerate(zip(input_sents_enc,target_sents_dec)): #enumerate : keep a count of iterations
   
    for t,char in enumerate(input_sents_enc):
        encoderInput_data[ i , t , char2int[char] ] = 1  #i=index of word, t=index of char, cha2int[char]=index of char in our char dict 
                                                            #assign all of that to 1 in encoder
    for t,char in enumerate(target_sents_dec):
        decoderInput_data[ i, t , char2int[char] ] = 1
        if t > 0 :
            decoderTarget_data[ i , t-1 , char2int[char] ] = 1 #as the word starts with \t and end with \n
   
print("Encoding COMPLETED...")  

Max Enc Len: 53
Max Dec Len: 47
CREATED ZERO VECTORS
(207479, 53, 76)
input_sents_enc 207479
target_sents_dec 207479
Encoding COMPLETED...


# **Test Data Preprocessing**

In [49]:
m2_list_sentance_test=[]
m2_list_operation_test=[]

m2_list_sentance_test.append(load_annotation('/content/QALB-Test2014.m2')[0])
m2_list_operation_test.append(load_annotation('/content/QALB-Test2014.m2')[1])


In [50]:
tokensTest =[]
i=0
for i in m2_list_sentance_test:
    tokensTest = tokanizer(m2_list_sentance_test)
print(tokensTest[3])
len(tokensTest)

['تعجبني', 'المظاهرات', 'في', 'حمص', 'هنا', 'وهناك', 'مثل', 'ما', 'راينا', 'في', 'دير', 'بعلبة', 'طبعا', 'كشف', 'النظام', 'السوري', 'عن', 'وجود', 'اماكن', 'للتعذيب', 'و', 'للتقطيع', 'طبعا', 'بالتاكيد', 'هذه', 'لمن', 'يفتح', 'محله', 'او', 'يجلس', 'في', 'بيته', 'ولا', 'ينزل', 'لمظاهرة', 'او', 'لمن', 'يقول', 'عاش', 'الوطن', 'ابيا', 'سالما', 'فبالفعل', 'واسفاه', 'على', 'من', 'تبع', 'هذه', 'الفورة', 'وصدقها', 'و', 'ساند', 'اذيالها']


968

In [51]:
test_correction=[]
test_input=[]
for i in range(len(tokensTest)):
  test_input.append((extract(m2_list_operation_test[0][i][0])[0]))
  test_correction.append((extract(m2_list_operation_test[0][i][0])[1]))

In [53]:
#applying the preproccesing &separating the words again
test_x=[]
for i in range(len(test_input)):
    input_sent =[process(x) for x in test_input[i]]
    for line in input_sent :
            test_x.append(line)
#print("\n".join(target[:4]))
print("Number of items:",len(test_x))
test_x[0:4]

Number of items: 10690


['لا زال', 'الشبيحه', 'ان', 'ارواح']

In [67]:
#applying the preproccesing &separating the words again
target_y=[]
for i in range(len(test_correction)):
    linesout =[process(x) for x in test_correction[i]]
    for line in linesout :
            target_y.append(line)    
#print("\n".join(target[:4]))
print("Number of items:",len(target_y))
target_y[0:4]

Number of items: 10690


['لازال', 'الشبيحة', 'أن', 'أرواح']

In [55]:
# create  dataset
test_y= [] 

for line in target_y:
        output = '\t' + line + '\n'
        test_y.append(output)

print(test_y[:4])

['\tلازال\n', '\tالشبيحة\n', '\tأن\n', '\tأرواح\n']


In [56]:
print("LEN OF input_sents:",len(test_x))
print("LEN OF target_sents:",len(test_y))
print("input_texts",test_x[:10])
print("target_texts",test_y[:10])
   

LEN OF input_sents: 10690
LEN OF target_sents: 10690
input_texts ['لا زال', 'الشبيحه', 'ان', 'ارواح', 'اقل', 'كلفه', 'اذا', 'المعادله', 'المهينه', 'ان']
target_texts ['\tلازال\n', '\tالشبيحة\n', '\tأن\n', '\tأرواح\n', '\tأقل\n', '\tكلفة\n', '\tإذا\n', '\tالمعادلة\n', '\tالمهينة\n', '\tأن\n']


In [57]:

max_enc_len_test = max([len(x) for x in test_x])

max_dec_len_test = max([len(x) for x in test_y])

print("max_enc_len_test:",max_enc_len_test)
print("max_dec_len_test:",max_dec_len_test)

num_samples = len(test_x)

encoderInput_datax = np.zeros( (num_samples , max_enc_len_test , len(char_set)),dtype='float32' )
decoderInput_datay = np.zeros( (num_samples , max_dec_len_test , len(char_set)+2),dtype='float32' )
decoderTarget_datay = np.zeros( (num_samples , max_dec_len_test , len(char_set)+2),dtype='float32' )
print("CREATED ZERO VECTORS")


#to cover the enumration issue 
test_x_enc = [] 
test_y_dec = [] 

for line in test_x:
        test_x_enc.append(line)

for line in test_y:
        test_y_dec.append(line)

print("input_sents_enc",len(test_x_enc))
print("test_y_dec",len(test_y_dec))
  

for i,(test_x_enc,test_y_dec) in enumerate(zip(test_x_enc,test_y_dec)): #enumerate : keep a count of iterations
   
    for t,char in enumerate(test_x_enc):
        encoderInput_datax[ i , t , char2int[char] ] = 1  #i=index of word, t=index of char, cha2int[char]=index of char in our char dict 
                                                            #assign all of that to 1 in encoder
    for t,char in enumerate(test_y_dec):
        decoderInput_datay[ i, t , char2int[char] ] = 1
        if t > 0 :
            decoderTarget_datay[ i , t-1 , char2int[char] ] = 1 #as the word starts with \t and end with \n
   
print("Encoding COMPLETED...")  


max_enc_len_test: 33
max_dec_len_test: 35
CREATED ZERO VECTORS
input_sents_enc 10690
test_y_dec 10690
Encoding COMPLETED...


# **THE MODEL**

In [22]:
#beginning of the model
from keras.models import Model
from keras.layers import Input,LSTM,Dense

In [23]:
#intializing the model
batch_size = 128
epochs = 200
latent_dim = 256  #embedding dimentions
num_enc_tokens =76
num_dec_tokens = 76 + 2 # includes \n \t
encoder_inputs = Input(shape=(None,num_enc_tokens))

#return_state: need to have its cell state initialized with previous time step while the weights are shared
encoder = LSTM(latent_dim,return_state=True) 
encoder_outputs , state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h,state_c]

In [24]:
decoder_inputs = Input(shape=(None,num_dec_tokens))
#return sequences return the hidden state output for each input time step.
#return state returns the hidden state output and cell state for the last input time step

decoder_lstm = LSTM(latent_dim,return_sequences=True,return_state=True) 
decoder_ouputs,_,_ = decoder_lstm(decoder_inputs,initial_state = encoder_states)

decoder_dense = Dense(num_dec_tokens, activation='softmax')
decoder_ouputs = decoder_dense(decoder_ouputs)

model = Model([encoder_inputs,decoder_inputs],decoder_ouputs)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy')
model.summary()

In [ ]:
#train the model
h=model.fit(
         [encoderInput_data,decoderInput_data],decoderTarget_data,
         epochs = epochs,
          batch_size = batch_size,
          )
model.save('s2s.h5')

Epoch 1/200
1621/1621 [==============================] - 34s 17ms/step - loss: 0.4299
Epoch 2/200
1621/1621 [==============================] - 27s 17ms/step - loss: 0.3577
Epoch 3/200
1621/1621 [==============================] - 28s 17ms/step - loss: 0.3443
Epoch 4/200
1621/1621 [==============================] - 27s 17ms/step - loss: 0.3223
Epoch 5/200
1621/1621 [==============================] - 27s 17ms/step - loss: 0.3035
Epoch 6/200
1621/1621 [==============================] - 27s 17ms/step - loss: 0.2886
Epoch 7/200
1621/1621 [==============================] - 27s 17ms/step - loss: 0.2732
Epoch 8/200
1621/1621 [==============================] - 27s 17ms/step - loss: 0.2592
Epoch 9/200
1621/1621 [==============================] - 27s 17ms/step - loss: 0.2453
Epoch 10/200
1621/1621 [==============================] - 27s 17ms/step - loss: 0.2307
Epoch 11/200
1621/1621 [==============================] - 27s 17ms/step - loss: 0.2174
Epoch 12/200
1621/1621 [============================

# **Validation Accuracy**

In [88]:
score = model.evaluate([encoderInput_datax,decoderInput_datay], decoderTarget_datay, batch_size=batch_size, verbose=0)

In [86]:
print("Validation accuracy: " ,score)

Validation accuracy:  0.7122483849525452


# **LOADING MODEL**

In [32]:
from keras.models import load_model
Model = load_model('/content/s2s.h5')

In [30]:
decoder_model = load_model('/content/decoder.h5')

In [31]:
encoder_model = load_model('/content/encoder.h5')

In [ ]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy')

In [71]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_dec_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, char2int['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = int2char[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_dec_len):
            stop_condition = True
            
        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_dec_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [75]:
#model testing

#encoder_model = Model(encoder_inputs,encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h,decoder_state_input_c]
decoder_outputs,state_h,state_c = decoder_lstm(
        decoder_inputs,initial_state = decoder_states_inputs
)
decoder_states = [state_h,state_c]
decoder_outputs = decoder_dense(decoder_outputs)

# decoder_model = Model(
#     [decoder_inputs] + decoder_states_inputs,
#     [decoder_outputs] + decoder_states
# ) 
#encoder_model.save('encoder.h5')
#decoder_model.save('decoder.h5')

for seq_index in range(100):
    input_seq = encoderInput_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Wrong sentence:', input_sents[seq_index])
    print('Corrected sentence:', decoded_sentence)
    print('Ground Truth:',target_sents[seq_index])

-
Wrong sentence: الى
Corrected sentence: إلى

Ground Truth: 	إلى

-
Wrong sentence: اكيد
Corrected sentence: أكيد

Ground Truth: 	أكيد

-
Wrong sentence: ان
Corrected sentence: أن

Ground Truth: 	أن

-
Wrong sentence: لحكام
Corrected sentence: لحكاما

Ground Truth: 	للحكام

-
Wrong sentence: ادناها
Corrected sentence: أدناها

Ground Truth: 	أدناها

-
Wrong sentence: .
Corrected sentence: 

Ground Truth: 	!

-
Wrong sentence: كذالك
Corrected sentence: كذلك

Ground Truth: 	كذلك

-
Wrong sentence: اخواننا
Corrected sentence: إخواننا

Ground Truth: 	إخواننا

-
Wrong sentence: الصينينة
Corrected sentence: الصينية

Ground Truth: 	الصينية

-
Wrong sentence: ثلاتة
Corrected sentence: ثلاثة

Ground Truth: 	ثلاثة

-
Wrong sentence: اشهر
Corrected sentence: أشهر

Ground Truth: 	أشهر

-
Wrong sentence: .
Corrected sentence: 

Ground Truth: 	!

-
Wrong sentence: الاسى
Corrected sentence: الأسي

Ground Truth: 	الأسى

-
Wrong sentence: الاليم
Corrected sentence: الأليم

Ground Truth: 	الأليم

-
Wron

# **Word Error Rate**

In [ ]:
!pip install jiwer

In [58]:
hypothesis=[]
ground_truth=[]
for seq_index in range(len(test_x)):
    input_seq = encoderInput_data[seq_index: seq_index + 1]
    decoded_sentence=decode_sequence(input_seq)
    hypothesis.append(decoded_sentence)
    ground_truth.append(target_sents[seq_index])

In [ ]:
from jiwer import wer
import jiwer
transformation = jiwer.Compose([
    jiwer.ToLowerCase(),
    jiwer.RemoveMultipleSpaces(),
    jiwer.RemoveWhiteSpace(replace_by_space=False),
    jiwer.SentencesToListOfWords(word_delimiter=" ")
]) 
error = wer(ground_truth, hypothesis,truth_transform=transformation,hypothesis_transform=transformation)

In [87]:
print("Word Error Rate : ",error)

Word Error Rate :  0.24087932647333957
